In [94]:
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/13.TrafficPrediction')
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df_train=pd.read_csv('train_aWnotuB.csv',parse_dates=['DateTime'])
df_test=pd.read_csv('test_BdBKkAj.csv',parse_dates=['DateTime'])

monthly_regression_params={1: [2.93, 18.59], 2: [0.9, 6.18], 3: [0.5, 8.65], 4: [0.17, 5.19]}

In [108]:
def get_vars(df):
    df['year']=df['DateTime'].dt.year
    df['month']=df['DateTime'].dt.month
    df['day']=df['DateTime'].dt.day
    df['hour']=df['DateTime'].dt.hour
    df['day_of_week']=df['DateTime'].dt.dayofweek
    df['flag_sunday']=(df['day_of_week']==6).astype(np.int64)
    df['flag_saturday']=(df['day_of_week']==5).astype(np.int64)
    df['flag_friday']=(df['day_of_week']==4).astype(np.int64)
    df['flag_monday']=(df['day_of_week']==0).astype(np.int64)
    df['flag_weekday']=(df['day_of_week']<=4).astype(np.int64)
    df['flag_junction_1']=(df['Junction']==1).astype(np.int64)
    df['flag_junction_2']=(df['Junction']==2).astype(np.int64)
    df['flag_junction_3']=(df['Junction']==3).astype(np.int64)
    df['flag_junction_4']=(df['Junction']==4).astype(np.int64)
    df['flag_last_day_of_month']=(df['day']==df['month'].map({1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31})).astype(np.int64)
    df['encoded_month']=12*df['year']+df['month']-24191
    df['encoded_day']=(df['DateTime']-date(2015,11,1)).dt.days
    slope=df['Junction'].map(lambda x: monthly_regression_params[x][0])
    intercept=df['Junction'].map(lambda x: monthly_regression_params[x][1])
    df['monthly_avg']=intercept+slope*df['encoded_month']
    return df

train=get_vars(df_train)
test=get_vars(df_test)



In [88]:
print mean_squared_error(train['monthly_avg'], train['Vehicles'])
train['Vehicles'].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])

96.5081680798


count    48120.000000
mean        22.009871
std         18.315982
min          1.000000
1%           2.000000
5%           4.000000
10%          6.000000
25%          9.000000
50%         15.000000
75%         29.000000
90%         53.000000
95%         70.000000
99%         70.000000
max         70.000000
Name: Vehicles, dtype: float64

In [110]:
def model_train(df,test):
    df=df.sort_values(by='DateTime')
    df_train1=df.head(int(0.85*len(df)))
    df_valid1=df.tail(len(df)-len(df_train1))
    model_cols=['monthly_avg','month','hour','day_of_week']+[x for x in df.columns.tolist() if 'flag' in x]
    regr = RandomForestRegressor(n_estimators=5, max_depth=5, random_state=0,min_samples_leaf=50,verbose=0)
    regr.fit(df_train1[model_cols],df_train1['Vehicles'])
    print pd.Series(regr.feature_importances_,index=model_cols).sort_values(ascending=False)
    df['score']=regr.predict(df[model_cols])
    df['error']=np.abs(df['score']-df['Vehicles'])
    df['error_rank']=np.round(df['error'].rank(pct=True),3)
    df.to_csv('chk1.csv')
    print np.sqrt(mean_squared_error(regr.predict(df_train1[model_cols]), df_train1['Vehicles']))
    print np.sqrt(mean_squared_error(regr.predict(df_valid1[model_cols]), df_valid1['Vehicles']))
    return np.round(regr.predict(test[model_cols]))
    
test['Vehicles']=model_train(train,test)
test[['ID','Vehicles']].to_csv('Model1.csv',index=False)

monthly_avg               0.819338
hour                      0.123669
day_of_week               0.026190
flag_weekday              0.020565
flag_junction_2           0.005142
month                     0.004228
flag_junction_3           0.000868
flag_last_day_of_month    0.000000
flag_junction_4           0.000000
flag_junction_1           0.000000
flag_monday               0.000000
flag_friday               0.000000
flag_saturday             0.000000
flag_sunday               0.000000
dtype: float64
5.94910161609
6.99847163461


In [ ]:
#Just for preserving the code
def model_train(df,test):
    df=df.sort_values(by='DateTime')
    df['rank']=df.groupby('Junction')['DateTime'].rank(ascending=True)
    df['total']=df['Junction'].map(df['Junction'].value_counts())
    df['Percentile']=1.0*df['rank']/df['total']
    df_train1=df[df['Percentile']<0.85]
    df_valid1=df[df['Percentile']>=0.85]
    model_cols=['monthly_avg','month','hour','day_of_week']+[x for x in df.columns.tolist() if 'flag' in x]
    print len(model_cols)
    print model_cols
    sm={}
    for name, group in df_train1.groupby('Junction'):
        regr = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=0,min_samples_leaf=50,verbose=0)
        if name==4:
            model_cols.remove('month')
        regr.fit(group[model_cols],group['Vehicles'])
        feature_importance=pd.Series(regr.feature_importances_,index=model_cols).sort_values(ascending=False)
        feature_importance=feature_importance.cumsum()
        new_cols=feature_importance[feature_importance<0.99].index
        regr = RandomForestRegressor(n_estimators=1, max_depth=10, random_state=0,min_samples_leaf=50,verbose=0)
        regr.fit(group[new_cols],group['Vehicles'])
        feature_importance=pd.Series(regr.feature_importances_,index=new_cols).sort_values(ascending=False)
        print feature_importance
        sm[name]=[new_cols,regr]
        
    all_scores=[pd.Series(sm[name][1].predict(group[sm[name][0]]),index=group['ID']) for name,group in df.groupby('Junction')]
    all_scores=np.round(pd.concat(all_scores))
    df['score']=df['ID'].map(all_scores)
    df['error']=np.abs(df['score']-df['Vehicles'])
    df['error_rank']=np.round(df['error'].rank(pct=True),3)
    df.to_csv('chk1.csv')
    print np.sqrt(mean_squared_error(df_train1['ID'].map(all_scores), df_train1['Vehicles']))
    print np.sqrt(mean_squared_error(df_valid1['ID'].map(all_scores), df_valid1['Vehicles']))
    test_scores=pd.concat([pd.Series(sm[name][1].predict(group[sm[name][0]]),index=group['ID']) for name,group in test.groupby('Junction')])
    test_scores=np.round(test_scores)
    regr = RandomForestRegressor(n_estimators=10, max_depth=10, random_state=0,min_samples_leaf=50,verbose=0)
    regr.fit(train[model_cols],train['Vehicles'])
    test['Vehicles']=np.round(regr.predict(test[model_cols]))
    test_scores=pd.Series(test['Vehicles'],index=test['ID'])
    return test_scores
    
test['Vehicles']=test['ID'].map(model_train(train,test))
test[['ID','Vehicles']].to_csv('Model1.csv',index=False)

In [1]:
imp_days=ins[['Junction','encoded_date','Vehicles']]
imp_days_prev['encoded_date']=imp_days['encoded_date']+1
imp_days=pd.merge(imp_days,imp_days_prev,on=['Junction','encoded_date'])
imp_days['Delta']=imp_days['Vehicles_x']/imp_days['Vehicles_y']-1

NameError: name 'ins' is not defined